In [1]:
from PIL import Image
import base64
import anthropic
from dotenv import load_dotenv
import os
from collections import defaultdict
import os
import random
from tqdm import tqdm

random.seed(42)
load_dotenv()


def resize_image(image_path, size):
    """resize image so that the largest edge is atmost size"""
    img = Image.open(image_path)
    width, height = img.size

    if width <= size and height <= size:
        return img

    if width > height:
        new_width = size
        new_height = int(height * (size / width))
    else:
        new_height = size
        new_width = int(width * (size / height))
    img = img.resize((new_width, new_height), Image.Resampling.LANCZOS)
    return img


def encode_image(image_path):
    img = resize_image(image_path, 512)
    temp_name = "temp.jpg"
    img.save(temp_name)
    with open(temp_name, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")


def construct_mcq(options, correct_option):
    correct_option_letter = None
    i = "a"
    mcq = ""

    for option in options:
        if option == correct_option:
            correct_option_letter = i
        mcq += f"{i}. {option}\n"
        i = chr(ord(i) + 1)

    mcq = mcq[:-1]
    return mcq, correct_option_letter


def add_row(content: list[dict], data: dict, i: int, with_answer=False) -> list[dict]:
    content.append(
        {
            "type": "text",
            "text": "Image " + str(i) + ": " + data["question"] + "\n" + data["mcq"],
        }
    )

    content.append(
        {
            "type": "image",
            "source": {
                "type": "base64",
                "media_type": "image/jpeg",
                "data": encode_image(data["image_path"]),
            },
        }
    )

    if with_answer:
        content.append(
            {
                "type": "text",
                "text": "Answer {}: ".format(i) + data["correct_option_letter"],
            }
        )
    else:
        content.append(
            {
                "type": "text",
                "text": "Answer {}: ".format(i),
            }
        )

    return content

In [2]:
FEWSHOT_JSON = "illusionVQA/comprehension/fewshot_labels.json"
FEWSHOT_IMAGE_DIR = "illusionVQA/comprehension/FEW_SHOTS/"
EVAL_JSON = "illusionVQA/comprehension/eval_labels.json"
EVAL_IMAGE_DIR = "illusionVQA/comprehension/EVAL/"

client = anthropic.Anthropic()
model_name = "claude-3-5-sonnet-20240620"

In [3]:
import json

with open(FEWSHOT_JSON) as f:
    fewshot_dataset = json.load(f)

for data in fewshot_dataset:
    data["image_path"] = FEWSHOT_IMAGE_DIR + data["image"]
    data["mcq"], data["correct_option_letter"] = construct_mcq(
        data["options"], data["answer"]
    )

In [4]:
with open(EVAL_JSON) as f:
    eval_dataset = json.load(f)

random.shuffle(eval_dataset)

In [5]:
category_count = defaultdict(int)
misc_cats = [
    "counting",
    "repeating pattern",
    "perspective",
    "occlusion",
    "angle constancy",
]

for data in eval_dataset:
    if data["image"] not in os.listdir(EVAL_IMAGE_DIR):
        print(data["image"], "not found")
        continue
    data["image_path"] = EVAL_IMAGE_DIR + data["image"]
    data["mcq"], data["correct_option_letter"] = construct_mcq(
        data["options"], data["answer"]
    )
    
    if data["category"] in misc_cats:
        data["category"] = "misc"
        
    category_count[data["category"]] += 1

print(category_count)
print(len(eval_dataset))

defaultdict(<class 'int'>, {'deceptive design': 37, 'impossible object': 134, 'misc': 19, 'color': 23, 'edited-scene': 21, 'size': 46, 'hidden': 45, 'real-scene': 64, 'angle illusion': 26, 'circle-spiral': 6, 'upside-down': 7, 'positive-negative space': 7})
435


### 0-shot

In [6]:
# content = [
#     {
#         "type": "text",
#         "text": "You'll be given an image, an instruction and some options. You have to select the correct one. Do not explain your reasoning. Answer with only the letter which corresponds to the correct option. Do not repeat the entire answer.",
#     }
# ]
# next_idx = 1

### 4-shot

In [6]:
content = [
    {
        "type": "text",
        "text": "You'll be given an image, an instruction and some choices. You have to select the correct one. Do not explain your reasoning. Answer with the option's letter from the given choices directly. Here are a few examples:",
    }
]


i = 1
for data in fewshot_dataset:
    content = add_row(content, data, i, with_answer=True)
    i += 1

content.append(
    {
        "type": "text",
        "text": "Now you try it!",
    }
)

next_idx = i

### Evaluation Loop

In [7]:
ytrue = []
ypred = []

In [9]:
import time

MAX_RETRIES = 2
for data in tqdm(eval_dataset):
    content_t = add_row(content.copy(), data, next_idx, with_answer=False)
    retries = MAX_RETRIES
    while retries:
        try:
            message = client.messages.create(
                model=model_name,
                max_tokens=1024,
                messages=[
                    {
                        "role": "user",
                        "content": content_t,
                    }
                ],
            )
            # print(message)
            claude_ans = message.content[0].text.lower().strip()
            break
        except Exception as e:
            print(e)
            retries -= 1
            time.sleep(30)
            if retries == 0:
                claude_ans = "Claude could not answer this question."
                print("retries exhausted")
                break

    answer = data["correct_option_letter"].strip()
    ytrue.append(answer)
    ypred.append(claude_ans)

100%|██████████| 435/435 [22:28<00:00,  3.10s/it]


In [10]:
for i in range(len(ypred)):
    if ypred[i] == "Claude could not answer this question.":
        print(i)

In [11]:
len(ytrue), len(ypred)

(435, 435)

### Result Analysis

In [12]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from collections import Counter

print(accuracy_score(ytrue, ypred))
print(confusion_matrix(ytrue, ypred))
print(classification_report(ytrue, ypred))

print(Counter(ytrue))
print(Counter(ypred))

0.664367816091954
[[79 14 12 10  1  0  0]
 [11 72 13 10  3  0  0]
 [12 17 69 11  0  0  0]
 [ 8  4  7 64  3  0  0]
 [ 0  4  2  1  4  0  0]
 [ 1  0  0  0  1  1  1]
 [ 0  0  0  0  0  0  0]]
              precision    recall  f1-score   support

           a       0.71      0.68      0.70       116
           b       0.65      0.66      0.65       109
           c       0.67      0.63      0.65       109
           d       0.67      0.74      0.70        86
           e       0.33      0.36      0.35        11
           f       1.00      0.25      0.40         4
           h       0.00      0.00      0.00         0

    accuracy                           0.66       435
   macro avg       0.58      0.48      0.49       435
weighted avg       0.67      0.66      0.66       435

Counter({'a': 116, 'c': 109, 'b': 109, 'd': 86, 'e': 11, 'f': 4})
Counter({'a': 111, 'b': 111, 'c': 103, 'd': 96, 'e': 12, 'h': 1, 'f': 1})


/home/salkhon/miniconda3/envs/general/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/salkhon/miniconda3/envs/general/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/salkhon/miniconda3/envs/general/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(r

In [13]:
import prettytable
from collections import defaultdict

table = prettytable.PrettyTable()
table.field_names = ["Category", "Total", "Wrong", "Accuracy"]

got_wrong_dict = defaultdict(int)
total_wrong = 0

for i in range(len(ypred)):
    if ypred[i] != ytrue[i]:
        got_wrong_dict[eval_dataset[i]["category"]] += 1
        total_wrong += 1


for k, v in got_wrong_dict.items():
    table.add_row([k, category_count[k], v, 1 - (v / category_count[k])])

table.add_row(
    [
        "Total",
        len(eval_dataset),
        total_wrong,
        1 - (total_wrong / len(eval_dataset)),
    ]
)


# sort by total
table.sortby = "Total"
table.reversesort = True
print(table)

+-------------------------+-------+-------+--------------------+
|         Category        | Total | Wrong |      Accuracy      |
+-------------------------+-------+-------+--------------------+
|          Total          |  435  |  146  | 0.6643678160919539 |
|    impossible object    |  134  |   49  | 0.6343283582089552 |
|        real-scene       |   64  |   27  |      0.578125      |
|           size          |   46  |   9   | 0.8043478260869565 |
|          hidden         |   45  |   25  | 0.4444444444444444 |
|     deceptive design    |   37  |   12  | 0.6756756756756757 |
|          color          |   23  |   3   | 0.8695652173913043 |
|       edited-scene      |   21  |   6   | 0.7142857142857143 |
|           misc          |   19  |   9   | 0.5263157894736843 |
|       upside-down       |   7   |   1   | 0.8571428571428572 |
| positive-negative space |   7   |   2   | 0.7142857142857143 |
|      circle-spiral      |   6   |   3   |        0.5         |
+------------------------

In [14]:
eval_dataset_copy = eval_dataset.copy()

print(len(eval_dataset_copy))
for i, data in enumerate(eval_dataset_copy):
    # map letter to option f
    if "BLOCK" in ypred[i]:
        data["vlm_answer"] = "BLOCK"
    else:
        try:
            data["vlm_answer"] = data["options"][ord(ypred[i]) - ord("a")]
        except Exception as e:
            data["vlm_answer"] = ypred[i]
            print(ypred[i])

435


In [15]:
import json

METRIC_SAVE_DIR = "../../results_and_evaluation/closed_source/results/"

with open(METRIC_SAVE_DIR + "illusionvqa_comprehension_claude_4shot_results.json", "w") as f:
    json.dump(eval_dataset_copy, f, indent=4)